In [1]:
pip install tensorflow

In [2]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, LeakyReLU, BatchNormalization, Reshape, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
import numpy as np
import matplotlib.pyplot as plt

In [3]:
# Define the Generator model
def build_generator(latent_dim):
    model = Sequential()
    model.add(Dense(256, input_dim=latent_dim))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(512))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(1024))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(784, activation='tanh'))
    model.add(Reshape((28, 28, 1)))
    return model

In [4]:
def build_discriminator(img_shape):
    model = Sequential()
    model.add(Flatten(input_shape=img_shape))
    model.add(Dense(512))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(256))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(1, activation='sigmoid'))
    return model

In [5]:
def build_gan(generator, discriminator):
    discriminator.trainable = False
    model = Sequential()
    model.add(generator)
    model.add(discriminator)
    return model


In [6]:
def train_gan(gan, generator, discriminator, data, latent_dim, epochs=100, batch_size=128):
    for epoch in range(epochs):
        for batch_idx in range(data.shape[0] // batch_size):
            # Generate noise data for the generator
            noise = np.random.normal(0, 1, size=(batch_size, latent_dim))

            # Generate fake images using the generator
            generated_images = generator.predict(noise)

            # Select a random batch of real images from the data
            real_images = data[np.random.randint(0, data.shape[0], batch_size)]

            # Create the training set for the discriminator
            X_train = np.concatenate([generated_images, real_images])

            # Labels for the discriminator (0: fake, 1: real)
            y_discriminator = np.zeros(2 * batch_size)
            y_discriminator[:batch_size] = 0.9  # One-sided label smoothing

            # Train the discriminator
            discriminator_loss = discriminator.train_on_batch(X_train, y_discriminator)

            # Create noise data again for training the generator
            noise = np.random.normal(0, 1, size=(batch_size, latent_dim))
            y_generator = np.ones(batch_size)

            # Train the generator
            generator_loss = gan.train_on_batch(noise, y_generator)

        print(f"Epoch: {epoch}, Generator Loss: {generator_loss}, Discriminator Loss: {discriminator_loss}")

        # Display generated images every 10 epochs
        if epoch % 10 == 0:
            plot_generated_images(generator, epoch)


In [7]:
def plot_generated_images(generator, epoch, examples=10, dim=(1, 10), figsize=(10, 1)):
    noise = np.random.normal(0, 1, size=(examples, latent_dim))
    generated_images = generator.predict(noise)
    plt.figure(figsize=figsize)
    for i in range(examples):
        plt.subplot(dim[0], dim[1], i+1)
        plt.imshow(generated_images[i].reshape(28, 28), interpolation='nearest', cmap='gray')
        plt.axis('off')
    plt.tight_layout()
    plt.savefig(f"generated_images_epoch_{epoch}.png")
    plt.close()


In [8]:
if __name__ == '__main__':
    # Load MNIST dataset
    (X_train, _), (_, _) = tf.keras.datasets.mnist.load_data()
    X_train = (X_train.astype(np.float32) - 127.5) / 127.5
    X_train = X_train.reshape(X_train.shape[0], 784)

11490434/11490434 [==============================] - 1s 0us/step
